In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# # Testing to ensure GPU is being utilized.

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np

import os
from zipfile import ZipFile

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, Dropout, GlobalMaxPooling2D, MaxPooling2D ,Flatten,BatchNormalization, Activation
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint


np.random.seed(42)
tf.random.set_seed(42)

# Read the data

In [ ]:
zippath="/content/drive/MyDrive/Gender_prediction/Dataset/Final_folder_gender.zip"

with ZipFile(zippath,"r") as data:
  data.extractall()
print("unzip done")

unzip done


In [ ]:
df_aug_trainset=pd.read_csv("/content/drive/MyDrive/Gender_prediction/csv/images_filenames_labels_train_gender.csv")
print(df_aug_trainset.shape)
df_aug_trainset.head()

(77114, 3)


,file,gender,gender_class
0,/content/Final_folder_gender/Women/68037.jpg,F,1
1,/content/Final_folder_gender/Men/43544.jpg,M,0
2,/content/Final_folder_gender/Women/10312.jpg,F,1
3,/content/Final_folder_gender/Men/51653.jpg,M,0
4,/content/Final_folder_gender/Men/27572.jpg,M,0


In [ ]:
df_test_refined=pd.read_csv("/content/drive/MyDrive/Gender_prediction/csv/images_filenames_labels_test_gender.csv")
print(df_test_refined.shape)
df_test_refined.head()

(33050, 3)


,file,gender,gender_class
0,/content/Final_folder_gender/Women/2_0_2_20161...,F,1
1,/content/Final_folder_gender/Women/59917.jpg,F,1
2,/content/Final_folder_gender/Women/3_1_4_20161...,F,1
3,/content/Final_folder_gender/Women/84964.jpg,F,1
4,/content/Final_folder_gender/Men/6261.jpg,M,0


# Create Tensorflow Dataset

In [ ]:
train_filename_tensor=tf.constant(list(df_aug_trainset["file"]))
train_label_tensor=tf.constant(list(df_aug_trainset["gender_class"]))

test_filename_tensor=tf.constant(list(df_test_refined["file"]))
test_label_tensor=tf.constant(list(df_test_refined["gender_class"]))

In [ ]:
num_classes = 2

def parse_function(filename, label):

    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)/255   # channels=1 to convert to grayscale, channels=3 to convert to RGB.
    image_resized = tf.image.resize(image_decoded, [224, 224])
    image_scaled=image_resized/255
    label = tf.one_hot(label, num_classes)

    return image_scaled, label

In [ ]:
train_tf_dataset=tf.data.Dataset.from_tensor_slices((train_filename_tensor,train_label_tensor)).map(parse_function).batch(256)
test_tf_dataset=tf.data.Dataset.from_tensor_slices((test_filename_tensor,test_label_tensor)).map(parse_function).batch(256)


# Define model architecture

In [ ]:
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.models import Sequential
from keras.models import Model

base_model=MobileNet( weights='imagenet',include_top=False,input_shape=(224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalMaxPooling2D()(x)
dense_1 = Dense(1024, activation='relu')(x)
dropout_1 = Dropout(0.5)(dense_1)  # Adding dropout layer with a dropout rate of 0.3
dense_2 = Dense(512, activation='relu')(dropout_1)
dropout_2 = Dropout(0.5)(dense_2)  # Adding another dropout layer with a dropout rate of 0.3
dense_3 = Dense(256, activation='relu')(dropout_2)
dropout_3 = Dropout(0.5)(x)  # Adding another dropout layer with a dropout rate of 0.3
dense_4=Dense(64, activation='relu')(dropout_3)
dropout_4 = Dropout(0.5)(x)
preds = Dense(2, activation='softmax')(dropout_4)

model=Model(inputs=base_model.input,outputs=preds)



In [ ]:
#freezing the initial layers
for layer in model.layers[:35]:
    layer.trainable=False
for layer in model.layers[35:]:
    layer.trainable=True

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [ ]:
#creating model checkpoint
tensorboard = TensorBoard(log_dir=f"/content/drive/MyDrive/Gender_prediction/log_final/")
checkpoint = ModelCheckpoint(filepath=f"/content/drive/MyDrive/Gender_prediction/log_final/model_checkpoint_gen_freezed.h5",
                             monitor='val_accuracy',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )

# Train the model

In [ ]:
history=model.fit(train_tf_dataset,batch_size=256,
                        validation_data=test_tf_dataset,
                        epochs=30, callbacks=[tensorboard,checkpoint],
                        shuffle=False)

Epoch 1/30
302/302 [==============================] - ETA: 0s - loss: 1.0612 - accuracy: 0.8077
Epoch 1: val_accuracy improved from -inf to 0.56880, saving model to /content/drive/MyDrive/Gender_prediction/log_final/model_checkpoint_gen_freezed.h5
302/302 [==============================] - 330s 997ms/step - loss: 1.0612 - accuracy: 0.8077 - val_loss: 0.7094 - val_accuracy: 0.5688
Epoch 2/30
302/302 [==============================] - ETA: 0s - loss: 0.2286 - accuracy: 0.9027
Epoch 2: val_accuracy improved from 0.56880 to 0.81909, saving model to /content/drive/MyDrive/Gender_prediction/log_final/model_checkpoint_gen_freezed.h5
302/302 [==============================] - 289s 958ms/step - loss: 0.2286 - accuracy: 0.9027 - val_loss: 0.4148 - val_accuracy: 0.8191
Epoch 3/30
302/302 [==============================] - ETA: 0s - loss: 0.1729 - accuracy: 0.9276
Epoch 3: val_accuracy improved from 0.81909 to 0.88811, saving model to /content/drive/MyDrive/Gender_prediction/log_final/model_checkp

# Visualise the training process

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

In [ ]:
# Plotting a line chart to visualize the loss and accuracy values by epochs.

import matplotlib.pyplot as plt
fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax = ax.ravel()

ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)

ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)

ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)

ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)

ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)

ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)

fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)


# Model evaluate

In [ ]:

final_cnn_score = model.evaluate(test_tf_dataset, verbose=1)


In [ ]:
from google.colab import runtime
runtime.unassign()